# Programming Notebook
*Author: Jacob Park*

## Graphs

A graph $G$ is a pair $(V, E)$ where $V$ is a set of vertices and $E$ is a set of edges.

### Implementation with Adjacency Lists

In [1]:
package graphs;

import java.util.*;

public class ListGraph {
    
    private final ArrayList<LinkedList<Integer>> adjacencyList;
    private final int vertices;
    
    public ListGraph(int vertices) {
        this.adjacencyList = new ArrayList<>(vertices);
        this.vertices = vertices;
        for (int vertex = 0; vertex < vertices; vertex++) {
            this.adjacencyList.add(new LinkedList<>());
        }
    }
    
    public void addEdge(int v, int w) {
        adjacencyList.get(v).add(w);
    }
    
    public int getVertices() {
        return vertices;
    }
    
    public Iterable<Integer> getAdjacents(int v) {
        return adjacencyList.get(v);
    }
    
}

graphs.ListGraph

### Implementation with Adjacency Matrix

In [2]:
package graphs;

import java.util.*;

public class MatrixGraph {
    
    private final boolean[][] adjacencyMatrix;
    private final int vertices;
    
    public MatrixGraph(int vertices) {
        this.adjacencyMatrix = new boolean[vertices][vertices];
        this.vertices = vertices;
    }
    
    public void addEdge(int v, int w) {
        adjacencyMatrix[v][w] = true;
    }
    
    public int getVertices() {
        return vertices;
    }
    
    public Iterable<Integer> getAdjacents(int v) {
        boolean[] adjacencyVector = adjacencyMatrix[v];
        LinkedList<Integer> adjacencyList = new LinkedList<>();
        for (int vertex = 0; vertex < adjacencyVector.length; vertex++) {
            if (adjacencyVector[vertex]) {
                adjacencyList.add(vertex);
            }
        }
        return adjacencyList;
    }
    
}

graphs.MatrixGraph

### Traversing with Depth-First Search

In [3]:
package graphs.dfs;

import graphs.*;

import java.util.*;
import java.util.function.*;

public class DepthFirstSearch {
    
    public static void depthFirstSearch(
        ListGraph graph, 
        int v, 
        Consumer<Integer> action) {
        Set<Integer> visitedSet = new HashSet<>();
        Stack<Integer> stack = new Stack<>();
        
        stack.push(v);
        
        while (!stack.isEmpty()) {
            int currentVertex = stack.pop();
            
            visitedSet.add(currentVertex);
            
            action.accept(currentVertex);
            
            Iterable<Integer> adjacents = graph.getAdjacents(currentVertex);
            for (int adjacentVertex : adjacents) {
                if (!visitedSet.contains(adjacentVertex)) {
                    stack.push(adjacentVertex);
                }
            }
        }
    }
    
}

graphs.dfs.DepthFirstSearch

In [4]:
package graphs.dfs;

import graphs.*;

ListGraph graph = new ListGraph(4);
graph.addEdge(0, 1);
graph.addEdge(0, 2);
graph.addEdge(1, 2);
graph.addEdge(2, 0);
graph.addEdge(2, 3);
graph.addEdge(3, 3);

DepthFirstSearch.depthFirstSearch(graph, 2, (vertex) -> System.out.println(vertex));

2
3
0
1


null

### Traversing with Breadth-First Search

In [5]:
package graphs.bfs;

import graphs.*;

import java.util.*;
import java.util.function.*;

public class BreadthFirstSearch {
    
    public static void breadthFirstSearch(
        ListGraph graph, 
        int v, 
        Consumer<Integer> action) {
        Set<Integer> visitedSet = new HashSet<>();
        Deque<Integer> queue = new ArrayDeque<>();
        
        queue.offer(v);
        
        while (!queue.isEmpty()) {
            int currentVertex = queue.poll();
            
            visitedSet.add(currentVertex);
            
            action.accept(currentVertex);
            
            Iterable<Integer> adjacents = graph.getAdjacents(currentVertex);
            for (int adjacentVertex : adjacents) {
                if (!visitedSet.contains(adjacentVertex)) {
                    queue.offer(adjacentVertex);
                }
            }
        }
    }
    
}

graphs.bfs.BreadthFirstSearch

In [6]:
package graphs.bfs;

import graphs.*;

ListGraph graph = new ListGraph(4);
graph.addEdge(0, 1);
graph.addEdge(0, 2);
graph.addEdge(1, 2);
graph.addEdge(2, 0);
graph.addEdge(2, 3);
graph.addEdge(3, 3);

BreadthFirstSearch.breadthFirstSearch(graph, 2, (vertex) -> System.out.println(vertex));

2
0
3
1


null

### Shortest Paths with Dijkstra's Algorithm ($O(|E| + |V|\log(|V|))$)

In [7]:
package graphs.dijkstra;

import graphs.*;

import java.util.*;

public class Dijkstra {

    private static class VertexDistancePair implements Comparable<VertexDistancePair> {

        private final int vertex;
        private final int distance;

        public VertexDistancePair(int vertex, int distance) {
            this.vertex = vertex;
            this.distance = distance;
        }

        public int getVertex() {
            return vertex;
        }

        public int getDistance() {
            return distance;
        }

        @Override
        public int compareTo(VertexDistancePair other) {
            return Integer.compare(this.distance, other.distance);
        }

    }
    
    private static List<Integer> buildPath(Map<Integer, Integer> vertexTo, int destination) {
        if (vertexTo.containsKey(destination)) {
            LinkedList<Integer> path = new LinkedList<>(Collections.singletonList(destination));

            int currentVertex = destination;
            while (vertexTo.containsKey(currentVertex)) {
                currentVertex = vertexTo.get(currentVertex);
                path.addFirst(currentVertex);
            }

            return path;
        } else {
            return Collections.emptyList();
        }
    }

    /*
     * Shortest Paths with Positive-Weights Only:
     *
     * 1. Assume all vertices are an infinite distance away from the source.
     * 2. Assume the source is zero distance away from the source.
     * 3. Perform a breadth-first search using a priority queue by distance 
     *    instead of a regular queue.
     * 4. For every unvisted adjacent vertex, relax its distance from 
     *    the source by taking the minimum of its current distance or 
     *    the distance from the source to the current vertex to 
     *    the adjacent vertex.
     *
     *    i.e. The shortest path can be greedily attained by choosing 
     *         to stay on the current path or to swap to a shorter path.
     */
    public static List<Integer> shortestPath(
        ListGraph graph,
        int[][] weights,
        int source,
        int destination) {
        Map<Integer, Integer> distanceTo = new HashMap<>();
        Map<Integer, Integer> vertexTo = new HashMap<>();
        Set<Integer> visitedSet = new HashSet<>();
        PriorityQueue<VertexDistancePair> priorityQueue = new PriorityQueue<>();

        for (int vertex = 0; vertex < graph.getVertices(); vertex++) {
            distanceTo.put(vertex, Integer.MAX_VALUE);
        }
        distanceTo.put(source, 0);

        priorityQueue.offer(new VertexDistancePair(source, distanceTo.get(source)));

        while (!priorityQueue.isEmpty()) {
            int minimumVertex = priorityQueue.poll().getVertex();

            visitedSet.add(minimumVertex);

            Iterable<Integer> adjacents = graph.getAdjacents(minimumVertex);
            for (int adjacentVertex : adjacents) {
                if (!visitedSet.contains(adjacentVertex)) {
                    int vDistance = distanceTo.get(minimumVertex);
                    int wDistance = distanceTo.get(adjacentVertex);
                    int edgeDistance = weights[minimumVertex][adjacentVertex];
                    if (wDistance > vDistance + edgeDistance) {
                        distanceTo.put(adjacentVertex, vDistance + edgeDistance);
                        vertexTo.put(adjacentVertex, minimumVertex);
                        priorityQueue.offer(new VertexDistancePair(adjacentVertex, distanceTo.get(adjacentVertex)));
                    }
                }
            }
        }

        return buildPath(vertexTo, destination);
    }

}

graphs.dijkstra.Dijkstra

In [8]:
package graphs.dijkstra;

import graphs.*;

import java.util.*;

ListGraph graph = new ListGraph(5);
graph.addEdge(0, 1);
graph.addEdge(0, 4);
graph.addEdge(1, 2);
graph.addEdge(1, 4);
graph.addEdge(2, 3);
graph.addEdge(3, 0);
graph.addEdge(3, 2);
graph.addEdge(4, 1);
graph.addEdge(4, 2);
graph.addEdge(4, 3);

int[][] weights = new int[5][5];
weights[0][1] = 10;
weights[0][4] = 5;
weights[1][2] = 1;
weights[1][4] = 2;
weights[2][3] = 4;
weights[3][0] = 7;
weights[3][2] = 6;
weights[4][1] = 3;
weights[4][2] = 9;
weights[4][3] = 2;

List<Integer> path = Dijkstra.shortestPath(graph, weights, 0, 2);

System.out.println(path.toString());

[0, 4, 1, 2]


null

### Shortest Paths with Bellman-Ford Algorithm ($O(|V||E|)$)

In [9]:
package graphs.bellman_ford;

import graphs.*;

import java.util.*;

public class BellmanFord {

    private static List<Integer> buildPath(Map<Integer, Integer> vertexTo, int destination) {
        if (vertexTo.containsKey(destination)) {
            LinkedList<Integer> path = new LinkedList<>(Collections.singletonList(destination));

            int currentVertex = destination;
            while (vertexTo.containsKey(currentVertex)) {
                currentVertex = vertexTo.get(currentVertex);
                path.addFirst(currentVertex);
            }

            return path;
        } else {
            return Collections.emptyList();
        }
    }

    /*
     * Shortest Paths with Negative-Weights Allowed:
     *
     * 1. Assume all vertices are an infinite distance away from the source.
     * 2. Assume the source is zero distance away from the source.
     * 3. For every vertex and its adjacent vertex, relax its distance from 
     *    the source by taking the minimum of its current distance or 
     *    the distance from the source to the current vertex to 
     *    the adjacent vertex.
     */
    public static List<Integer> shortestPath(
        ListGraph graph,
        int[][] weights,
        int source,
        int destination) {
        Map<Integer, Integer> distanceTo = new HashMap<>();
        Map<Integer, Integer> vertexTo = new HashMap<>();

        for (int vertex = 0; vertex < graph.getVertices(); vertex++) {
            distanceTo.put(vertex, Integer.MAX_VALUE);
        }
        distanceTo.put(source, 0);

        for (int vertex = 0; vertex < graph.getVertices(); vertex++) {
            Iterable<Integer> adjacents = graph.getAdjacents(vertex);
            for (int adjacentVertex : adjacents) {
                int vDistance = distanceTo.get(vertex);
                int wDistance = distanceTo.get(adjacentVertex);
                int edgeDistance = weights[vertex][adjacentVertex];
                if (wDistance > vDistance + edgeDistance) {
                    distanceTo.put(adjacentVertex, vDistance + edgeDistance);
                    vertexTo.put(adjacentVertex, vertex);
                }
            }
        }

        return buildPath(vertexTo, destination);
    }

}

graphs.bellman_ford.BellmanFord

In [10]:
package graphs.bellman_ford;

import graphs.*;

import java.util.*;

ListGraph graph = new ListGraph(5);
graph.addEdge(0, 1);
graph.addEdge(0, 4);
graph.addEdge(1, 2);
graph.addEdge(1, 4);
graph.addEdge(2, 3);
graph.addEdge(3, 0);
graph.addEdge(3, 2);
graph.addEdge(4, 1);
graph.addEdge(4, 2);
graph.addEdge(4, 3);

int[][] weights = new int[5][5];
weights[0][1] = 10;
weights[0][4] = 5;
weights[1][2] = 1;
weights[1][4] = 2;
weights[2][3] = 4;
weights[3][0] = 7;
weights[3][2] = 6;
weights[4][1] = 3;
weights[4][2] = 9;
weights[4][3] = 2;

List<Integer> path = BellmanFord.shortestPath(graph, weights, 0, 2);

System.out.println(path.toString());

[0, 4, 1, 2]


null